# Convolutional Neural Networks

## Project: Write an Algorithm for Landmark Classification

### Install Prerequisites

To run the app in the notebook environment, you must first install the required packages by executing the two cells below. **Make sure to restart the kernel after running each cell.**

> Note: Restarting the kernel ensures that all installed dependencies are properly loaded into the environment.

In [ ]:
# Please restart the notebook kernel after running this cell.
!pip install -r requirements.txt 

In [ ]:
!pip install ipywidgets

In [ ]:
# Please restart the notebook kernel after running this cell as well.
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter nbextension enable --py widgetsnbextension

### A Simple App

In this notebook we build a very simple app that uses our exported model.

> <img src="static_images/icons/noun-info-2558213.png" alt="?" style="width:25px"/> Note how we are not importing anything from our source code (we do not use any module from the ``src`` directory). This is because the exported model, differently from the model weights, is a standalone serialization of our model and therefore it does not need anything else. You can ship that file to anybody, and as long as they can import ``torch``, they will be able to use your model. This is very important for releasing pytorch models to production.

### Test Your App
Go to a search engine for images (like Google Images) and search for images of some of the landmarks, like the Eiffel Tower, the Golden Gate Bridge, Machu Picchu and so on. Save a few examples locally, then upload them to your app to see how your model behaves!

The app will show the top 5 classes that the model think are most relevant for the picture you have uploaded

In [ ]:
from ipywidgets import VBox, Button, FileUpload, Output, Label, IntSlider
from PIL import Image
from IPython.display import display
import io
import numpy as np
import torchvision
import torch
import torchvision.transforms as T
import os
import json

# Check if model file exists
model_path = "checkpoints/transfer_exported.pt"
if os.path.exists(model_path):
    try:
        learn_inf = torch.jit.load(model_path)
        print("Model loaded successfully")
        
        # Try to get class names from the model
        try:
            class_names = learn_inf.get_class_names()
            print(f"Class names loaded from model: {len(class_names)} classes")
        except:
            # If model doesn't have class names, try to load from JSON or data loaders
            class_names_path = "checkpoints/class_names.json"
            if os.path.exists(class_names_path):
                with open(class_names_path, 'r') as f:
                    class_names = json.load(f)
                print(f"Class names loaded from JSON: {len(class_names)} classes")
            else:
                # Fallback: try to get from data loaders
                try:
                    from src.data import get_data_loaders
                    data_loaders = get_data_loaders(batch_size=32)
                    class_names = data_loaders["train"].dataset.classes
                    print(f"Class names loaded from data: {len(class_names)} classes")
                except:
                    # Last resort: create dummy class names
                    class_names = [f"Class_{i}" for i in range(50)]
                    print(f"Using dummy class names: {len(class_names)} classes")
                    
    except Exception as e:
        print(f"Error loading model: {e}")
        learn_inf = None
        class_names = []
else:
    print(f"Model file not found at {model_path}")
    learn_inf = None
    class_names = []

def on_click_classify(change):
    if learn_inf is None:
        print("Model not loaded. Cannot classify.")
        return
        
    if len(btn_upload.data) == 0:
        return  # No file uploaded yet

    # Load image that has been uploaded
    try:
        fn = io.BytesIO(btn_upload.data[-1])
        img = Image.open(fn).convert("RGB")
    except Exception as e:
        print(f"Error loading image: {e}")
        return

    # Let's clear the previous output (if any)
    out_pl.clear_output()

    # Display the image
    with out_pl:
        ratio = img.size[0] / img.size[1]
        c = img.copy()
        c.thumbnail([ratio * 200, 200])
        display(c)

    # Transform the image
    try:
        # Use the same preprocessing as in your Predictor class
        transform = T.Compose([
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize(mean=[0.4638, 0.4725, 0.4687], std=[0.2699, 0.2706, 0.3018])  # Use your dataset stats
        ])

        timg = transform(img).unsqueeze(0)  # Add batch dimension

        # Move model and input to CPU or GPU
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        timg = timg.to(device)
        learn_inf.to(device)

        # Call the model
        with torch.no_grad():
            softmax = learn_inf(timg).cpu().numpy().squeeze()

        # Get class indices sorted by probability (highest first)
        idxs = np.argsort(softmax)[::-1]

        # Loop over the top 5 predictions
        for i in range(5):
            p = softmax[idxs[i]]
            if i < len(class_names):
                landmark_name = class_names[idxs[i]]
                labels[i].value = f"{landmark_name} (prob: {p:.4f})"
            else:
                labels[i].value = f"Class {idxs[i]} (prob: {p:.4f})"
                
    except Exception as e:
        print(f"Error during classification: {e}")
        for label in labels:
            label.value = "Error occurred"

# Widgets setup
btn_upload = FileUpload(accept='image/*', multiple=False)
btn_run = Button(description="Classify")
btn_run.on_click(on_click_classify)
out_pl = Output()
out_pl.clear_output()

labels = [Label() for _ in range(5)]

wgs = [Label("Please upload a picture of a food item"), btn_upload, btn_run, out_pl]
wgs.extend(labels)

# Display the interactive app
VBox(wgs)

## (Optional) Standalone App or Web App

You can run this notebook as a standalone app on your computer by following these steps:

1. Download this notebook in a directory on your machine
2. Download the model export (for example, ``checkpoints/transfer_exported.pt``) in a subdirectory called ``checkpoints`` within the directory where you save the app.ipynb notebook
3. Install voila if you don't have it already (``pip install voila``)
4. Run your app: ``voila app.ipynb --show_tracebacks=True``
5. Customize your notebook to make your app prettier and rerun voila

You can also deploy this app as a website using Binder: https://voila.readthedocs.io/en/stable/deploy.html#deployment-on-binder

# Create Your Submission Archive

Now that you are done with your project, please run the following cell. It will generate a file containing all the code you have written, as well as the notebooks. Please submit that file to complete your project

In [ ]:
!python src/create_submit_pkg.py